### Commands

In [1]:
from firecloud import api as firecloud_api
import glob
import fc_interface

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [172]:
### Delete all cohorts
cohort_metadata = pd.read_table('cohort_files/cohort_names_dictionary.txt', header=None, names=['name', 'code'])
cohort_codes = cohort_metadata.code.tolist()
for c in cohort_codes:
    fc_interface.delete_sample_set(namespace, workspace, c)

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

In [336]:
### Prepare cohorts for export to FC
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [358]:
# # Delete cohorts
# cohort_codes = pd.read_table('cohort_files/cohort_names_dictionary.txt', header=None)
# for coh_code in cohort_codes.loc[:, 1].tolist():
#     res = fc_interface.delete_sample_set(namespace, workspace, coh_code)

In [338]:
# ### Upload metadata for given batch
# fc_interface.update_batch_metadata('TSCA21', \
#                                   '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
#                                       paths_to_samples_info, \
#                                         namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [361]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [359]:
# Cohorts
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [66]:
### Create panel of normals from batch
pon = fc_interface.create_panel_of_normals_from_batch(batch_id, paths_to_samples_info, N=20).shape

In [78]:
### Create panel of normals from batch
for i, row in batches_info.iterrows():
    pon, name = fc_interface.create_panel_of_normals_from_batch(row['tsca_id'], paths_to_samples_info, N=20)
    fc_interface.upload_pon(pon, name, namespace, workspace)

In [ ]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [33]:
### Prepare cohorts for export to FC
cohorts = fc_interface.\
            prepare_cohorts_for_metadata_export(paths_to_samples_info, google_bucket_id, \
                                               blacklist=["CCLF_AA1012-Tumor-SM-F67DF"])

cohorts_sample_set_metadata = "cohort_files/fc_upload_sample_set_cohorts.txt"
res = fc_interface.upload_entities_from_tsv(namespace, workspace, cohorts_sample_set_metadata)

In [8]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [34]:
### Cohort of all tumors
res = fc_interface.upload_cohort_all_tumors(paths_to_samples_info, google_bucket_id, \
                                      'Cum_Tumors_22_all', namespace, workspace, ['DW039-Tumor-SM-DB2IF'])

In [12]:
# fc_interface.merge_walkupseq_files(paths_to_samples_info)

(1045, 14)

---

---

## Refined: New Batch Workflow

In [5]:
### PARAMETERS
tsca_id = "TSCA22"
tsca_num = "22"
blacklist = ["DW039-Tumor-SM-DB2IF"]
path_to_batch_samples_info = batches_info.loc[batches_info.tsca_id==tsca_id, 'path_to_samples_info'].item()

In [29]:
### ALL SAMPLES
## Prepare
# fc_interface.prepare_patients_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_sample_set_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_samples_for_metadata_export(path_to_batch_samples_info, tsca_id, google_bucket_id)

## Export
# fc_interface.export_batch_metadata_to_fc(tsca_id, namespace, workspace)

In [39]:
### PANEL OF NORMALS BATCH
## Prepare 
# pon, name = fc_interface.create_panel_of_normals_from_batch(tsca_id, paths_to_samples_info, N=20)

# Export
# fc_interface.upload_pon(pon, name, namespace, workspace)

In [13]:
### PANEL OF NORMALS CUMULATIVE
## Prepare
# batches_info = pd.read_excel(paths_to_samples_info)
# PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_22_all')

# Export
# fc_interface.upload_pon(PoN, 'Cum_PoN_22_all', namespace, workspace)

Creating panel of 163 normals


<Response [200]>

In [183]:
### COHORT SAMPLE SETS
## Prepare
# cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_samples_info, \
#                                                            google_bucket_id, blacklist, tsca_id, \
#                                                           namespace, workspace)

# ## Export
# res = fc_interface.upload_cohorts(namespace, workspace, tsca_id)

In [231]:
# ### PAIRS
# ## Prepare
# clean_pair_list = fc_interface.prepare_pairs_for_metadata_exports(paths_to_samples_info, tsca_id, google_bucket_id, blacklist)

# ## Export
# fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

FileNotFoundError: File b'/xchip/clf/seq_data/processed_for_fh/tsca1213_201610_SN0106956/tsca1213_201610_SN0106956.import_samples.txt' does not exist

In [6]:
clean_pair_list = fc_interface.prepare_pairs_for_metadata_exports(paths_to_samples_info, tsca_id, google_bucket_id, blacklist)

In [8]:
clean_pair_list.shape

(179, 7)

In [244]:
# ### PAIRSETS
# ## Prepare
# # all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)

# # ### Upload pairsets
# fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
# fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

<Response [200]>

In [6]:
### CUMULATIVE COHORT OF ALL SAMPLES
res = fc_interface.upload_cohort_all_samples(paths_to_samples_info, google_bucket_id, \
                                            'Cum_22_all', namespace, workspace, blacklist)

In [ ]:
### CUMULATIVE PANEL OF NORMALS

In [ ]:
### CREATE COHORTS PAIRSETS

In [35]:
all_samples = fc_interface.get_all_samples_with_cohort(tsca_id, paths_to_samples_info, google_bucket_id)
# Create list of pairs
pairs_list = fc_interface.create_pairs_list(all_samples)
# Filter blacklist
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

# # Samples in new batch
# samples_new_batch = pd.read_table("%s/fc_upload_sample_set_tsca_%s.txt"%(tsca_id, tsca_id))
# # Sample IDs in new batch
# new_batch_sample_ids = samples_new_batch.sample_id.tolist()

# # Only upload new pairs. 
# # New pairs are pairs that have a sample from the new batch as either a sample or control.
# clean_pairs_list = clean_pairs_list[ clean_pairs_list['case_sample_id'].isin(new_batch_sample_ids) | \
#                                    clean_pairs_list['control_sample_id'].isin(new_batch_sample_ids)]

In [41]:
all_samples[['entity:sample_id', 'cohort_code']]

,entity:sample_id,cohort_code
0,AA33-Normal-SM-D4L4D,COHORT_AA
1,AA34-Tumor-SM-D4L25,COHORT_AA
2,AA36-Tumor-SM-D4L2H,COHORT_AA
3,AA39-Tumor-SM-D4L3Q,COHORT_AA
4,AA40-Tumor-SM-D4L42,COHORT_AA
5,AA24-Tumor-SM-DHZ8W,COHORT_AA
6,AA25-Tumor-SM-DHZ99,COHORT_AA
7,AA33-Normal-SM-DHZ9L,COHORT_AA
8,AA33-Tumor-SM-DHZ9X,COHORT_AA
9,AA38-Tumor-SM-DHZ98,COHORT_AA


In [42]:
pairs_with_cohort = pd.merge(clean_pairs_list, all_samples[['entity:sample_id', 'cohort_code']], \
         left_on='case_sample_id', right_on='entity:sample_id')

In [44]:
pairs_with_cohort.head(2)

,entity:pair_id,case_sample_id,control_sample_id,participant_id,match_type,case_sample_tsca_id,control_sample_tsca_id,entity:sample_id,cohort_code
0,AA34-Tumor-SM-D4L25_NA_TN,AA34-Tumor-SM-D4L25,NA,AA34,tumor_normal,TSCA1213,NA,AA34-Tumor-SM-D4L25,COHORT_AA
1,AA34-Tumor-SM-D4L25_NA_TP,AA34-Tumor-SM-D4L25,NA,AA34,tumor_primary,TSCA1213,NA,AA34-Tumor-SM-D4L25,COHORT_AA


In [45]:
pairs_with_cohort_clean = pairs_with_cohort[['cohort_code', 'entity:pair_id']] \
    .rename(columns={'entity:pair_id': 'pair_id', 'cohort_code': 'membership:pair_set_id'})

In [47]:
pairs_with_cohort_clean.shape

(1950, 2)

In [29]:
all_samples.head(2)

,entity:sample_id,participant_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code,bam_filename,bai_filename,clean_bai_file_capture,tsca_id,Collection,cohort_code
0,AA33-Normal-SM-D4L4D,AA33,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA33N_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59KF,SM-D4L4D,Normal,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA33N_3D_p5,NaN,NDNA,1_AA33N_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,1_AA33N_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA1213,Cancer Cell Line Factory (CCLF) / Andy Aguirre...,COHORT_AA
1,AA34-Tumor-SM-D4L25,AA34,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA34T_3D_p4,TSCA Rapid Cancer Detection Panel v2,SM-D59I2,SM-D4L25,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA34T_3D_p4,Metastatic,TM,1_AA34T_3D_p4_HVLVWBCXX.1.aligned.duplicates_m...,1_AA34T_3D_p4_HVLVWBCXX.1.aligned.duplicates_m...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA1213,Cancer Cell Line Factory (CCLF) / Andy Aguirre...,COHORT_AA


In [59]:
cohort_pairsets.head(2)

,membership:pair_set_id,pair_id
0,COHORT_AA,AA34-Tumor-SM-D4L25_NA_TN
1,COHORT_AA,AA34-Tumor-SM-D4L25_NA_TP


In [52]:
cohort_pairsets = fc_interface.prepare_cohort_pairsets_for_metadata_exports\
    (tsca_id, blacklist, paths_to_samples_info, google_bucket_id)

In [54]:
res = fc_interface.upload_cohort_pairsets(namespace, workspace, cohort_pairsets)

In [57]:
res.json()

{'causes': [{'causes': [],
   'message': 'pair EH015-Tumor-SM-D4L1Z_EH015-Normal-SM-D4L2J_TN not found',
   'source': 'rawls',
   'stackTrace': []},
  {'causes': [],
   'message': 'pair PEDS167-Tumor-SM-DHZA3_PEDS167-Normal-SM-D4L3M_TP not found',
   'source': 'rawls',
   'stackTrace': []},
  {'causes': [],
   'message': 'pair CCLF_PEDS1025-Tumor-SM-F29S9_CCLF_PEDS1025-Normal-SM-EUVFO_TN not found',
   'source': 'rawls',
   'stackTrace': []},
  {'causes': [],
   'message': 'pair PEDS195-Tumor-SM-E7S28_PEDS195-Tumor-SM-DHZ9O_TP not found',
   'source': 'rawls',
   'stackTrace': []},
  {'causes': [],
   'message': 'pair PEDS196-Tumor-SM-E7RZA_PEDS196-Normal-SM-DNULT_TN not found',
   'source': 'rawls',
   'stackTrace': []},
  {'causes': [],
   'message': 'pair PEDS063-Tumor-SM-GBJ9W_PEDS063-Normal-SM-F67EX_TN not found',
   'source': 'rawls',
   'stackTrace': []},
  {'causes': [],
   'message': 'pair CCLF_PEDS1021-Tumor-SM-F67F8_CCLF_PEDS1021-Tumor-SM-E7S1A_TP not found',
   'source': 'r